In [1]:
# Connect to google drive (where the data is, to access it):
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 1. Install dependencies


In [ ]:
! pip install transformers #

In [3]:
# Classes for storing individual sentences:

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, label=None): #text_b=None,
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        # self.text_b = text_b
        self.label = label

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, attention_mask, token_type_ids, label_id, e1_mask, e2_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.label_id = label_id
        self.e1_mask = e1_mask
        self.e2_mask = e2_mask

        #add enitity position and entity mask for BERT
        # self.e11_p = e11_p
        # self.e12_p = e12_p
        # self.e21_p = e21_p
        # self.e22_p = e22_p
        # self.e1_mask = e1_mask
        # self.e2_mask = e2_mask
        
    def print_contents(self):
        print(self.input_ids,self.attention_mask,self.token_type_ids , self.label_id,
        self.e1_mask, self.e2_mask)

In [4]:
import os

def get_label(data_dir):
    return [label.strip() for label in open(os.path.join(data_dir, "label.txt"), "r", encoding="utf-8")]

In [5]:
rel_label = get_label('/content/gdrive/My Drive/NLP/semEval_Data/')
# rel_label

In [6]:
rel_label

['Other',
 'Cause-Effect(e1,e2)',
 'Cause-Effect(e2,e1)',
 'Instrument-Agency(e1,e2)',
 'Instrument-Agency(e2,e1)',
 'Product-Producer(e1,e2)',
 'Product-Producer(e2,e1)',
 'Content-Container(e1,e2)',
 'Content-Container(e2,e1)',
 'Entity-Origin(e1,e2)',
 'Entity-Origin(e2,e1)',
 'Entity-Destination(e1,e2)',
 'Entity-Destination(e2,e1)',
 'Component-Whole(e1,e2)',
 'Component-Whole(e2,e1)',
 'Member-Collection(e1,e2)',
 'Member-Collection(e2,e1)',
 'Message-Topic(e1,e2)',
 'Message-Topic(e2,e1)']

In [7]:
# Functions for reading in the data:

import csv
import sys 

def read_tsv(input_file, quotechar=None):
    """Reads a tab separated value file."""
    with open(input_file, "r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
        lines = []
        for line in reader:
            lines.append(line)
        return lines
        # for line in reader:
        #     if sys.version_info[0] == 2:
        #         line = list(cell for cell in line)
        #     lines.append(line)
        # return lines
      
def create_examples(lines, set_type):
    examples = []
    for (i, line) in enumerate(lines):

        guid = "%s-%s" % (set_type, i)
        text_a = line[1]
        label = rel_label.index(line[0])
        examples.append(InputExample(guid=guid, text_a=text_a, label=label))
    return examples

def get_train_examples(data_dir):
    return create_examples(
        read_tsv(os.path.join(data_dir, "train.tsv")), "train")
    

def get_test_examples(data_dir):
    return create_examples(
        read_tsv(os.path.join(data_dir, "test.tsv")), "test")

# 2. Read in the data and convert to features

In [8]:
from transformers import BertConfig, BertTokenizer

# Configuration parameters:
use_entity_indicator=True
max_seq_len=128

tokenizer = BertTokenizer.from_pretrained(
        'bert-base-uncased', do_lower_case=True)

n_labels = 19

# Create special tokens for beginning and end of entities
ADDITIONAL_SPECIAL_TOKENS = ["<e1>", "</e1>", "<e2>", "</e2>"]

# add special tokens to tokenizer
tokenizer.add_special_tokens({"additional_special_tokens": ADDITIONAL_SPECIAL_TOKENS})

# labels = [str(i) for i in range(n_labels)]
# labels = get_label()


4

In [9]:
def convert_examples_to_features(
    examples,
    max_seq_len,
    tokenizer,
    cls_token="[CLS]",
    cls_token_segment_id=0,
    sep_token="[SEP]",
    pad_token=0,
    pad_token_segment_id=0,
    sequence_a_segment_id=0,
    add_sep_token=False,
    mask_padding_with_zero=True,
):
    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        # print(f'tokens_a: {tokens_a}')

        e11_p = tokens_a.index("<e1>")  # the start position of entity1
        e12_p = tokens_a.index("</e1>")  # the end position of entity1
        e21_p = tokens_a.index("<e2>")  # the start position of entity2
        e22_p = tokens_a.index("</e2>")  # the end position of entity2

        # Replace the token
        tokens_a[e11_p] = "$"
        tokens_a[e12_p] = "$"
        tokens_a[e21_p] = "#"
        tokens_a[e22_p] = "#"

        # Add 1 because of the [CLS] token
        e11_p += 1
        e12_p += 1
        e21_p += 1
        e22_p += 1

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        if add_sep_token:
            special_tokens_count = 2
        else:
            special_tokens_count = 1
        if len(tokens_a) > max_seq_len - special_tokens_count:
            tokens_a = tokens_a[: (max_seq_len - special_tokens_count)]

        tokens = tokens_a
       
        if add_sep_token:
            tokens += [sep_token]

        token_type_ids = [sequence_a_segment_id] * len(tokens)

        tokens = [cls_token] + tokens
        token_type_ids = [cls_token_segment_id] + token_type_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
        attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_len - len(input_ids)
        input_ids = input_ids + ([pad_token] * padding_length)
        attention_mask = attention_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
        token_type_ids = token_type_ids + ([pad_token_segment_id] * padding_length)

        # e1 mask, e2 mask
        e1_mask = [0] * len(attention_mask)
        e2_mask = [0] * len(attention_mask)

        for i in range(e11_p, e12_p + 1):
            e1_mask[i] = 1
        for i in range(e21_p, e22_p + 1):
            e2_mask[i] = 1

        assert len(input_ids) == max_seq_len, "Error with input length {} vs {}".format(len(input_ids), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(
            len(attention_mask), max_seq_len
        )
        assert len(token_type_ids) == max_seq_len, "Error with token type length {} vs {}".format(
            len(token_type_ids), max_seq_len
        )

        label_id = int(example.label)

        features.append(
            InputFeatures(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                label_id=label_id,
                e1_mask=e1_mask,
                e2_mask=e2_mask,
            )
        )

    return features

In [10]:
import os

# Get the training data from the data folder, hosted on google drive:
data_folder = '/content/gdrive/My Drive/NLP/semEval_Data/'
examples = get_train_examples(data_folder)
features = convert_examples_to_features(
    examples, max_seq_len, tokenizer)

*Convert* the features to tensors and make a tensor data set

In [11]:
import torch 
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler,TensorDataset
 
 # Convert to Tensors and build dataset
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
all_e1_mask = torch.tensor([f.e1_mask for f in features], dtype=torch.long)  # add e1 mask
all_e2_mask = torch.tensor([f.e2_mask for f in features], dtype=torch.long)  # add e2 mask

all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

dataset = TensorDataset(
    all_input_ids,
    all_attention_mask,
    all_token_type_ids,
    all_label_ids,
    all_e1_mask,
    all_e2_mask,
)

# 3. Preparing the model

In [12]:
# Configuration parameters:

# batch size (low to save memory):
per_gpu_train_batch_size = 16
n_gpu = torch.cuda.device_count()

# the base BERT model (smaller, to save memory)
pretrained_model_name='bert-base-uncased'

# parameters for gradient descent:
max_steps=-1
gradient_accumulation_steps=1 

# Number of training epochs:
num_train_epochs=5

# Name of task for Bert:
task_name = 'semeval'

# hyperparameter for regularization
l2_reg_lambda=5e-3
local_rank=-1
no_cuda=False

train_batch_size = per_gpu_train_batch_size * \
        max(1, n_gpu)

# For sampling during the training:
train_sampler = RandomSampler(dataset)
train_dataloader = DataLoader(
        dataset, sampler=train_sampler, batch_size=train_batch_size)

# total number of steps for training:
t_total = len(train_dataloader) // gradient_accumulation_steps * num_train_epochs

# 4. Load the Bert customized for relation extraction

In [13]:
import torch
import torch.nn as nn
from transformers import BertModel, BertPreTrainedModel

class FCLayer(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.1, use_activation=True):
        super(FCLayer, self).__init__()
        self.use_activation = use_activation
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(input_dim, output_dim)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.dropout(x)
        if self.use_activation:
            x = self.tanh(x)
        return self.linear(x)


class MyBert(BertPreTrainedModel):
    def __init__(self, config):
        super(MyBert, self).__init__(config)
        self.bert = BertModel(config=config)  # Load pretrained bert

        self.num_labels = config.num_labels

        self.cls_fc_layer = FCLayer(config.hidden_size, config.hidden_size)
        self.entity_fc_layer = FCLayer(config.hidden_size, config.hidden_size)
        self.label_classifier = FCLayer(
            config.hidden_size * 3,
            config.num_labels,
            use_activation=False,
        )



    @staticmethod
    def entity_average(hidden_output, e_mask):
        """
        Average the entity hidden state vectors (H_i ~ H_j)
        :param hidden_output: [batch_size, j-i+1, dim]
        :param e_mask: [batch_size, max_seq_len]
                e.g. e_mask[0] == [0, 0, 0, 1, 1, 1, 0, 0, ... 0]
        :return: [batch_size, dim]
        """
        e_mask_unsqueeze = e_mask.unsqueeze(1)  # [b, 1, j-i+1]
        length_tensor = (e_mask != 0).sum(dim=1).unsqueeze(1)  # [batch_size, 1]

        # [b, 1, j-i+1] * [b, j-i+1, dim] = [b, 1, dim] -> [b, dim]
        sum_vector = torch.bmm(e_mask_unsqueeze.float(), hidden_output).squeeze(1)
        avg_vector = sum_vector.float() / length_tensor.float()  # broadcasting
        return avg_vector

    def forward(self, input_ids, attention_mask, token_type_ids, labels, e1_mask, e2_mask):
        outputs = self.bert(
            input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids
        )  # sequence_output, pooled_output, (hidden_states), (attentions)
        sequence_output = outputs[0]
        pooled_output = outputs[1]  # [CLS]

        # Average
        e1_h = self.entity_average(sequence_output, e1_mask)
        e2_h = self.entity_average(sequence_output, e2_mask)

        # Dropout -> tanh -> fc_layer (Share FC layer for e1 and e2)
        pooled_output = self.cls_fc_layer(pooled_output)
        e1_h = self.entity_fc_layer(e1_h)
        e2_h = self.entity_fc_layer(e2_h)

        # Concat -> fc_layer
        concat_h = torch.cat([pooled_output, e1_h, e2_h], dim=-1)
        logits = self.label_classifier(concat_h)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        # Softmax
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

            outputs = (loss,) + outputs

        return outputs 

In [14]:
bert_config = BertConfig.from_pretrained(
            pretrained_model_name,
            num_labels=n_labels,
            finetuning_task=task_name,
            id2label={str(i): label for i, label in enumerate(rel_label)},
            label2id={label: i for i, label in enumerate(rel_label)},
        )
        # self.model = MyBert.from_pretrained(args.model_name_or_path, config=self.config, args=args)

        # Load the model:
model = MyBert.from_pretrained(
        pretrained_model_name, config=bert_config)

# GPU or CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing MyBert: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing MyBert from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MyBert from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MyBert were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls_fc_layer.linear.weight', 'cls_fc_layer.l

MyBert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

### **Training**

In [15]:
# Prepare optimizer and schedule (linear warmup and decay)

from transformers import AdamW, get_linear_schedule_with_warmup

# Hyperparameters for the optimizer:
max_grad_norm = 1.0
learning_rate=2e-5
adam_epsilon=1e-8
warmup_steps=0
weight_decay=0.9


no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# Load optimizer and scheduler:
optimizer = AdamW(optimizer_grouped_parameters,
                  lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)

In [16]:
# Prepare for trainig:
from tqdm import tqdm, trange
import random
import numpy as np

#  Random seed for reproducability
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

global_step = 0
tr_loss, logging_loss = 0.0, 0.0
model.zero_grad()
train_iterator = trange(int(num_train_epochs),
                        desc="Epoch", disable=local_rank not in [-1, 0])



Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
# Loops through the training set for a few epochs and backpropagate

# Collect the loss values:
loss_values = []

seed = 123456
set_seed(seed)

for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration",
                          disable=local_rank not in [-1, 0])
    
    # For each epoch,  split into batches and train!

    for step, batch in enumerate(epoch_iterator):
        model.train()
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2],
                  'labels':      batch[3],
                  'e1_mask': batch[4],
                  'e2_mask': batch[5],
                  }

        outputs = model(**inputs)
        # model outputs are always tuple in transformers
        
        loss = outputs[0]

        # Collect the loss:
        loss_values.append(loss)
        
        if n_gpu > 1:
            loss = loss.mean()  
            # mean() to average on multi-gpu parallel training
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
        
        # Back propagate
        loss.backward()
        torch.nn.utils.clip_grad_norm_(
            model.parameters(), max_grad_norm)

        tr_loss += loss.item()
        if (step + 1) % gradient_accumulation_steps == 0:

            # Take a step! 
            optimizer.step()
            scheduler.step()              
            # Update learning rate schedule
            model.zero_grad()
            global_step += 1

        if max_steps > 0 and global_step > max_steps:
            # We're done!
            epoch_iterator.close()
            break
    if max_steps > 0 and global_step > max_steps:
        # We're done!
        train_iterator.close()
        break

In [18]:
# Save the trained model:
# torch.save(model.state_dict(), '/content/gdrive/My Drive/NLP/semEval_Data/semEvalModel_train')

In [19]:
# Metrics for evaluation (accuracy, f1 score),  from the official script for SemEval task-8
def acc_and_f1(preds, labels, average='macro'):
    acc = simple_accuracy(preds, labels)
    f1 = f1_score(y_true=labels, y_pred=preds, average=average)
    return {"acc": acc,
        "f1": f1,
        "acc_and_f1": (acc + f1) / 2}
    
def compute_metrics(task_name, preds, labels):
    assert len(preds) == len(labels)
    return acc_and_f1(preds, labels)

def simple_accuracy(preds, labels):
    return (preds == labels).mean()

In [20]:
# Evaluation

def evaluate(model, tokenizer, prefix=""):
    '''
    Reads the test set, makes predictions on it, saves the predictions
    returns the predictions / truth and accuracy+f1 score.
    '''
    # Loop to handle MNLI double evaluation (matched, mis-matched)

    # What kind of task it was, for BERT:
    eval_task = task_name

    # Save the evaluation metrics into results:
    results = {}

    # Load the test set and convert to features and to tensors:
    examples1 = get_test_examples('/content/gdrive/My Drive/NLP/semEval_Data/')
    features = convert_examples_to_features(
        examples1, max_seq_len, tokenizer, "classification", use_entity_indicator)

    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_e1_mask = torch.tensor([f.e1_mask for f in features], dtype=torch.long)  # add e1 mask
    all_e2_mask = torch.tensor([f.e2_mask for f in features], dtype=torch.long)  # add e2 mask

    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

    eval_dataset = TensorDataset(
        all_input_ids,
        all_attention_mask,
        all_token_type_ids,
        all_label_ids,
        all_e1_mask,
        all_e2_mask,
    )

    # eval_dataset = TensorDataset(all_input_ids, all_input_mask,all_segment_ids, all_label_ids, all_e1_mask, all_e2_mask)

    # Size of batch per GPU:
    eval_batch_size = per_gpu_eval_batch_size * \
        max(1, n_gpu)

    # Sample and load data:
    eval_sampler = SequentialSampler(
        eval_dataset) 
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=eval_batch_size)

  # Eval!
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None

    # Loop through the test set, batch by batch:

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2],
                      'labels':      batch[3],
                      'e1_mask': batch[4],
                      'e2_mask': batch[5],
                      }
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1

        # Extract the predictions from the model's output:
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs['labels'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(
                out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)
            
    # Get the loss, prediction and results:
    eval_loss = eval_loss / nb_eval_steps
    preds = np.argmax(preds, axis=1)


    result = compute_metrics(eval_task, preds, out_label_ids)
    results.update(result)
    
    # Write results to file:
    output_eval_file = "/content/gdrive/My Drive/NLP/semEval_Data/results_semEval.txt"
    with open(output_eval_file, "w", encoding="utf-8") as writer:
         for idx, pred in enumerate(preds):
             writer.write("{}\t{}\n".format(8001 + idx, rel_label[pred]))
                
    return result, preds, out_label_ids

In [21]:
from sklearn.metrics import matthews_corrcoef, f1_score

In [22]:
per_gpu_eval_batch_size=4

result = evaluate(model,tokenizer)
result

Evaluating: 100%|██████████| 680/680 [00:24<00:00, 28.27it/s]


({'acc': 0.8476260581523739,
  'acc_and_f1': 0.8275153747015158,
  'f1': 0.8074046912506576},
 array([17,  6,  4, ..., 14,  5,  8]),
 array([17,  6,  4, ..., 14,  5, 12]))

**Using the official scorer for evaluation**

In [26]:
cd './gdrive/My Drive/NLP/semEval_Data/'


/content/gdrive/My Drive/NLP/semEval_Data


In [36]:
!bash test_eval.sh

the results is saved in off_result.txt


In [37]:
cat off_result.txt

<<< (2*9+1)-WAY EVALUATION (USING DIRECTIONALITY)>>>:

Confusion matrix:
        C-E1 C-E2 C-W1 C-W2 C-C1 C-C2 E-D1 E-D2 E-O1 E-O2 I-A1 I-A2 M-C1 M-C2 M-T1 M-T2 P-P1 P-P2  _O_ <-- classified as
      +-----------------------------------------------------------------------------------------------+ -SUM- skip ACTUAL
 C-E1 | 126    2    0    0    0    0    0    0    0    0    0    0    0    0    1    0    0    1    4 |  134    0  134
 C-E2 |   1  182    0    0    0    0    0    0    3    0    0    0    0    0    0    0    1    0    7 |  194    0  194
 C-W1 |   0    0  137    2    1    1    0    0    0    0    4    1    3    1    1    0    1    0   10 |  162    0  162
 C-W2 |   0    0    2  130    0    2    0    0    0    0    0    5    0    2    3    0    0    0    6 |  150    0  150
 C-C1 |   0    0    2    0  138    0    7    0    0    0    1    0    0    0    0    0    0    0    5 |  153    0  153
 C-C2 |   0    0    0    1    0   36    0    0    0    1    0    0    0    1    0    0   

In [59]:
# ff = '8001	Message-Topic(e1,e2)'
# ff.split('\t')[1]

'Message-Topic(e1,e2)'

In [88]:
predictions = []

with open('/content/gdrive/My Drive/NLP/semEval_Data/results_semEval.txt') as f:
  for l in f.readlines():
    predictions.append(l.split('\t')[1].strip())

In [ ]:
# check some correct predictions
correct_sentences = []
with open(r'/content/gdrive/My Drive/NLP/semEval_Data/test.tsv') as f:
  correct = set()
  i = 0
  for l in f.readlines():
    if predictions[i] in l.split(' ')[0]:
      # print(predictions[i])
      # print(l.split('\t')[1])
      correct_sentences.append(l)
      correct.add((l, predictions[i]))
    i += 1

In [92]:
correct_sentences[1]

'Product-Producer(e2,e1)\tThe <e1> company </e1> fabricates plastic <e2> chairs </e2>.\n'

**Displaying Visualization with displacy**

In [ ]:
!python -m spacy download en

In [93]:
import spacy
from spacy import displacy

In [102]:
nlp = spacy.load('en_core_web_sm')

doc = nlp(correct_sentences[0])
displacy.render(doc, style='ent', jupyter=True)
displacy.render(doc, style='dep', jupyter=True, options={'distance': 100})


In [103]:
doc = nlp(correct_sentences[500])
displacy.render(doc, style='ent', jupyter=True)
displacy.render(doc, style='dep', jupyter=True, options={'distance': 100})

In [104]:
doc = nlp(correct_sentences[45])
displacy.render(doc, style='ent', jupyter=True)
displacy.render(doc, style='dep', jupyter=True, options={'distance': 100})

### **TODO**

#### 1. PORT CODE INTO PYTORCH LIGHTNING

#### 2. REPLACE FC LAYER WITH RNN
